# Logistic Regression


In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
import re

# To be able to import the file
import sys
pathname="/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/DSA4262-frontasticfour/scripts"
if pathname not in sys.path:
    sys.path.append(pathname)
    
import getData as gd

In [2]:
import importlib
importlib.reload(gd)

<module 'getData' from '/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/DSA4262-frontasticfour/scripts/getData.py'>

## Prepare raw data

In [3]:
df = pd.read_pickle("./test.pkl")

In [4]:
df.head(2)

,transcript,position,k-mer bases,f1_stats,f2_stats,f3_stats,f4_stats,f5_stats,f6_stats,f7_stats,f8_stats,f9_stats
0,ENST00000000233,244,AAGACCA,"[0.008264378378378385, 0.00199, 0.00432, 0.006...","[4.223783783783786, 1.77, 3.07, 3.73, 4.66, 13.4]","[123.70270270270271, 102.0, 123.0, 125.0, 126....","[0.009373243243243247, 0.00232, 0.0054, 0.0079...","[7.382162162162164, 1.04, 4.67, 6.65, 10.2, 15.7]","[125.91351351351351, 111.0, 124.0, 126.0, 128....","[0.007344918918918917, 0.00232, 0.00398, 0.005...","[4.386989189189191, 0.773, 1.95, 3.44, 5.68, 1...","[80.57027027027029, 73.1, 79.0, 80.5, 82.0, 88.3]"
1,ENST00000000233,261,CAAACTG,"[0.006609244186046515, 0.00199, 0.00432, 0.005...","[3.2164244186046504, 0.919, 2.4075, 2.88, 3.70...","[109.6813953488372, 98.3, 108.0, 110.0, 112.0,...","[0.006812906976744187, 0.00166, 0.0043025, 0.0...","[3.2265348837209307, 0.789, 2.48, 3.0, 3.67, 1...","[107.88953488372093, 96.1, 106.0, 108.0, 110.0...","[0.007710290697674419, 0.00232, 0.004410000000...","[3.0165988372093038, 0.715, 2.1675, 2.66, 3.49...","[94.29069767441861, 88.6, 92.8, 94.1, 95.725, ..."


In [5]:
df.columns

Index(['transcript', 'position', 'k-mer bases', 'f1_stats', 'f2_stats',
       'f3_stats', 'f4_stats', 'f5_stats', 'f6_stats', 'f7_stats', 'f8_stats',
       'f9_stats'],
      dtype='object')

In [6]:
df = df[['transcript', 'position',
       'k-mer bases', 'f1_stats', 'f2_stats', 'f3_stats', 'f4_stats',
       'f5_stats', 'f6_stats', 'f7_stats', 'f8_stats', 'f9_stats']]

In [7]:
df.head(2)

,transcript,position,k-mer bases,f1_stats,f2_stats,f3_stats,f4_stats,f5_stats,f6_stats,f7_stats,f8_stats,f9_stats
0,ENST00000000233,244,AAGACCA,"[0.008264378378378385, 0.00199, 0.00432, 0.006...","[4.223783783783786, 1.77, 3.07, 3.73, 4.66, 13.4]","[123.70270270270271, 102.0, 123.0, 125.0, 126....","[0.009373243243243247, 0.00232, 0.0054, 0.0079...","[7.382162162162164, 1.04, 4.67, 6.65, 10.2, 15.7]","[125.91351351351351, 111.0, 124.0, 126.0, 128....","[0.007344918918918917, 0.00232, 0.00398, 0.005...","[4.386989189189191, 0.773, 1.95, 3.44, 5.68, 1...","[80.57027027027029, 73.1, 79.0, 80.5, 82.0, 88.3]"
1,ENST00000000233,261,CAAACTG,"[0.006609244186046515, 0.00199, 0.00432, 0.005...","[3.2164244186046504, 0.919, 2.4075, 2.88, 3.70...","[109.6813953488372, 98.3, 108.0, 110.0, 112.0,...","[0.006812906976744187, 0.00166, 0.0043025, 0.0...","[3.2265348837209307, 0.789, 2.48, 3.0, 3.67, 1...","[107.88953488372093, 96.1, 106.0, 108.0, 110.0...","[0.007710290697674419, 0.00232, 0.004410000000...","[3.0165988372093038, 0.715, 2.1675, 2.66, 3.49...","[94.29069767441861, 88.6, 92.8, 94.1, 95.725, ..."


In [8]:
# Set the path the the raw data and data labels
path_to_data="/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/data/data.json"
path_to_labels="/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/data/data.info"

getData = gd.getData(path_to_data, path_to_labels)
label_df = getData.get_labels()

In [9]:
tdf = getData.get_data(num_entries=10, return_df=False)

In [10]:
# Merge the two sources
df = pd.merge(df, label_df, how='left', left_on=("transcript", "position"), 
             right_on=("transcript_id", "transcript_position"))

In [11]:
df = df.filter(regex=".*_stats|label")
df.head(2)

,f1_stats,f2_stats,f3_stats,f4_stats,f5_stats,f6_stats,f7_stats,f8_stats,f9_stats,label
0,"[0.008264378378378385, 0.00199, 0.00432, 0.006...","[4.223783783783786, 1.77, 3.07, 3.73, 4.66, 13.4]","[123.70270270270271, 102.0, 123.0, 125.0, 126....","[0.009373243243243247, 0.00232, 0.0054, 0.0079...","[7.382162162162164, 1.04, 4.67, 6.65, 10.2, 15.7]","[125.91351351351351, 111.0, 124.0, 126.0, 128....","[0.007344918918918917, 0.00232, 0.00398, 0.005...","[4.386989189189191, 0.773, 1.95, 3.44, 5.68, 1...","[80.57027027027029, 73.1, 79.0, 80.5, 82.0, 88.3]",0
1,"[0.006609244186046515, 0.00199, 0.00432, 0.005...","[3.2164244186046504, 0.919, 2.4075, 2.88, 3.70...","[109.6813953488372, 98.3, 108.0, 110.0, 112.0,...","[0.006812906976744187, 0.00166, 0.0043025, 0.0...","[3.2265348837209307, 0.789, 2.48, 3.0, 3.67, 1...","[107.88953488372093, 96.1, 106.0, 108.0, 110.0...","[0.007710290697674419, 0.00232, 0.004410000000...","[3.0165988372093038, 0.715, 2.1675, 2.66, 3.49...","[94.29069767441861, 88.6, 92.8, 94.1, 95.725, ...",0


In [12]:
feature_cols = list(filter(lambda x : re.search("stats", x) is not None, df.columns))

In [13]:
df = df.copy()
for idx, col in enumerate(feature_cols):
    df[[f"{idx+1}_{x}" for x in range(1, 7)]] = df.apply(lambda x : x[col], 
                                                           result_type="expand", 
                                                          axis=1)
df = df[df.columns.difference(feature_cols)]
tdf = df.head(5).copy()

In [14]:
tdf

,1_1,1_2,1_3,1_4,1_5,1_6,2_1,2_2,2_3,2_4,...,8_4,8_5,8_6,9_1,9_2,9_3,9_4,9_5,9_6,label
0,0.008264,0.00199,0.004320,0.00697,0.01100,0.0339,4.223784,1.770,3.0700,3.73,...,3.440,5.6800,15.50,80.570270,73.1,79.000,80.5,82.000,88.3,0
1,0.006609,0.00199,0.004320,0.00564,0.00797,0.0222,3.216424,0.919,2.4075,2.88,...,2.660,3.4900,14.10,94.290698,88.6,92.800,94.1,95.725,103.0,0
2,0.007570,0.00232,0.004400,0.00631,0.00963,0.0299,2.940541,1.280,2.1900,2.65,...,1.910,2.4400,6.85,89.364324,84.4,88.400,89.2,90.100,96.2,0
3,0.010620,0.00232,0.006475,0.00902,0.01340,0.0370,6.476350,1.220,4.6275,5.73,...,2.135,2.5525,6.49,89.154000,81.4,86.975,89.9,91.225,95.7,0
4,0.010701,0.00199,0.005770,0.00896,0.01390,0.0478,6.415051,1.150,5.4650,6.52,...,4.160,4.8975,8.81,85.178788,77.6,83.625,85.4,86.900,90.5,0


## Obtain Training Data

In [37]:
label_df['label'].value_counts(normalize=True)

0    0.955063
1    0.044937
Name: label, dtype: float64

In [15]:
# data
X = np.array(df[df.columns.difference(['label'])])
y = np.array(df['label'])

In [16]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

## Fitting Logistic Regression Model & Prediction

In [17]:
LogReg = LogisticRegression(solver="liblinear", penalty="l2")
LogReg.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

## Evaluation

In [18]:
y_pred = LogReg.predict(X_test)
ridge_LogReg_report = classification_report(y_test, y_pred)
print(ridge_LogReg_report)

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       240
           1       0.20      0.09      0.13        11

    accuracy                           0.94       251
   macro avg       0.58      0.54      0.55       251
weighted avg       0.93      0.94      0.93       251



In [19]:
res = LogReg.predict_proba(X_test)
# Prediction scores for each class [0,1]
res[1,]

array([0.94113856, 0.05886144])

# Comparison wiht dummy classifier

In [20]:
# Predict everything as the majority class
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
y_pred_dummy=dummy_clf.predict(X_test)

In [21]:
dummy_report = classification_report(y_test, y_pred_dummy)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

## Summarize report

In [22]:
with open("test_report.txt", "w") as f:
    f.write(f"Logistic Ridge Regression summary report : \n\n{ridge_LogReg_report}\n\n")
    f.write(f"Dummy Classifier (Majority class) summary report : \n\n{dummy_report}")
    

## Make Predictions

In [23]:
df_orig = pd.read_pickle("./test.pkl")
df = df_orig.copy()

In [24]:
feature_cols = list(filter(lambda x : re.search("stats", x) is not None, df.columns))
for idx, col in enumerate(feature_cols):
    df[[f"{idx+1}_{x}" for x in range(1, 7)]] = df.apply(lambda x : x[col], 
                                                           result_type="expand", 
                                                          axis=1)
df = df[df.columns.difference(feature_cols)]

In [25]:
df

,1_1,1_2,1_3,1_4,1_5,1_6,2_1,2_2,2_3,2_4,...,8_6,9_1,9_2,9_3,9_4,9_5,9_6,k-mer bases,position,transcript
0,0.008264,0.00199,0.004320,0.00697,0.011000,0.0339,4.223784,1.770,3.0700,3.73,...,15.50,80.570270,73.1,79.000,80.5,82.000,88.3,AAGACCA,244,ENST00000000233
1,0.006609,0.00199,0.004320,0.00564,0.007970,0.0222,3.216424,0.919,2.4075,2.88,...,14.10,94.290698,88.6,92.800,94.1,95.725,103.0,CAAACTG,261,ENST00000000233
2,0.007570,0.00232,0.004400,0.00631,0.009630,0.0299,2.940541,1.280,2.1900,2.65,...,6.85,89.364324,84.4,88.400,89.2,90.100,96.2,GAAACAG,316,ENST00000000233
3,0.010620,0.00232,0.006475,0.00902,0.013400,0.0370,6.476350,1.220,4.6275,5.73,...,6.49,89.154000,81.4,86.975,89.9,91.225,95.7,AGAACAT,332,ENST00000000233
4,0.010701,0.00199,0.005770,0.00896,0.013900,0.0478,6.415051,1.150,5.4650,6.52,...,8.81,85.178788,77.6,83.625,85.4,86.900,90.5,AGGACAA,368,ENST00000000233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.009711,0.00232,0.005380,0.00856,0.013325,0.0292,8.399429,2.370,6.4725,8.20,...,8.57,91.417143,85.4,90.025,91.5,92.675,99.4,AGAACTA,1758,ENST00000081029
997,0.009670,0.00232,0.006000,0.00831,0.012300,0.0256,3.508592,1.480,2.4300,2.86,...,3.46,86.121127,83.2,84.850,85.7,87.300,90.1,GAAACCC,1771,ENST00000081029
998,0.008702,0.00232,0.004980,0.00697,0.009980,0.0292,8.563488,3.260,6.9150,8.72,...,3.22,84.869767,79.3,83.550,84.8,86.550,89.6,AGAACCA,118,ENST00000084795
999,0.008231,0.00232,0.005640,0.00762,0.010500,0.0242,2.855965,1.770,2.3000,2.57,...,5.46,88.084211,80.5,85.400,87.4,90.600,95.7,AAAACAA,233,ENST00000084795


In [26]:
idx_cols = ["transcript", "position", "k-mer bases"]

In [27]:
X = np.array(df[df.columns.difference(idx_cols)])

In [28]:
X.shape

(1001, 54)

In [29]:
pred = LogReg.predict_proba(X)

In [30]:
pred = pred[:, 1]

In [31]:
df = df[idx_cols].copy()

In [32]:
df['scores'] = pred

In [33]:
df

,transcript,position,k-mer bases,scores
0,ENST00000000233,244,AAGACCA,0.001449
1,ENST00000000233,261,CAAACTG,0.001387
2,ENST00000000233,316,GAAACAG,0.003650
3,ENST00000000233,332,AGAACAT,0.030765
4,ENST00000000233,368,AGGACAA,0.000004
...,...,...,...,...
996,ENST00000081029,1758,AGAACTA,0.002717
997,ENST00000081029,1771,GAAACCC,0.003245
998,ENST00000084795,118,AGAACCA,0.001678
999,ENST00000084795,233,AAAACAA,0.002264


In [34]:
df = df.drop("k-mer bases", axis = 1)

In [35]:
df.columns = ["transcript_id", "transcript_position", "score"]

In [36]:
df.to_csv("some.csv", index=False)